In [1]:
import copy
import re
from collections import defaultdict

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Topics

In [22]:
topics = pd.read_csv("../topic_words.csv.zip")
topics.head()

,meta,topic,weight,type,word
0,env,Климат,5,0,ch4
1,env,Климат,5,0,co2
2,env,Климат,5,0,ghg protocol
3,env,Климат,5,0,scope 1
4,env,Климат,5,0,scope 2


In [23]:
topics["topic"].value_counts()
pass

In [24]:
re_dict = {}
re_advanced_dict = {}
sheets_dict = {}
sheets_advanced_dict = {}
base_dict = {}
advanced_dict = {}
importance_dict = {}

for idx, row in topics.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        sheets_advanced_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
        advanced_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    if row["type"] == 1:
        sheets_advanced_dict[row["topic"]][row["word"]] = row["weight"]
        advanced_dict[row["topic"]][row["word"]] = 0
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [25]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

In [26]:
for key, val in sheets_advanced_dict.items():
    re_advanced_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

# Expert

In [27]:
regex_expert = re.compile("(" + "(?!\w)|".join(topics["word"].unique()) + "(?!\w))")

In [28]:
label2idx = {topic: i for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2label = {i: topic for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2word = {i: word for i, word in enumerate(topics["word"].tolist())}
word2idx = defaultdict(list)

for i, word in enumerate(topics["word"].tolist()):
    word2idx[word].append(i)

In [29]:
topics_vectors = np.zeros((topics["topic"].nunique(), topics.shape[0]), dtype=int)

for topic, topic_words in topics.groupby("topic")["word"].agg(list).items():
    for word in topic_words:
        for idx in word2idx[word]:
            topics_vectors[label2idx[topic], idx] = 1

In [30]:
df = pd.read_csv("../paragraph_max_values_30.csv", index_col=0).rename(
    columns={"Топик, определенный экспертом": "true_expert"}
)
df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text,max_topic_cos,max_score_cos,second_max_topic_cos,...,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,Совпадение топиков абзацев,true_expert,"Эффективность COS по первому топику, %","Эффективность экспертного метода по первому топику, %","Эффективность COS по второму топику, %","Эффективность экспертного метода по второму топику, %"
806093.0,ПАО «Северсталь»,2017,Металлургическая и горнодобывающая,ОУР,2275.0,Ответственность в цепочке поставок «Северсталь...,ответственность цепочка поставка северсталь ра...,Безопасность и охрана труда,0.634112,Закупки и антикоррупция,...,Безопасность и охрана труда,-558.0,Качество и безопасность продукт,-570.0,топики совпадают,Безопасность и охрана труда,1,1,FALSE,FALSE
947952.0,ПАО «ЛУКОЙЛ»,2014,Нефтегазовая,ОУР,359.0,информации. Работает Служба по контролю и внут...,информация работать служба контроль внутренний...,Безопасность и охрана труда,0.083624,NaN,...,Отчетность и прозрачность,-73.0,Лидерство,-95.0,разные топики,Отчетность и прозрачность,FALSE,1,FALSE,FALSE
1035964.0,ПАО «Полюс»,2018,Металлургическая и горнодобывающая,ОУР,1509.0,Предотвращение травматизма Предотвращение прои...,предотвращение травматизм предотвращение произ...,Безопасность и охрана труда,0.473050,Риски,...,Безопасность и охрана труда,-570.0,Качество и безопасность продукт,-578.0,топики совпадают,Безопасность и охрана труда,1,1,FALSE,FALSE
243643.0,ПАО «Северсталь»,2015,Металлургическая и горнодобывающая,ОУР,1666.0,С 2016 года Компания планирует перейти на комп...,с год компания планировать перейти комплексный...,Безопасность и охрана труда,0.323875,Риски,...,Безопасность и охрана труда,-125.0,Риски,-127.0,топики совпадают,Безопасность и охрана труда,1,1,FALSE,FALSE
773067.0,АО «Атомредметзолото»,2017,Металлургическая и горнодобывающая,ИО,2594.0,• в рамках перехода к международным стандартам...,рамка переход международный стандарт управлени...,Безопасность и охрана труда,0.430482,Качество и безопасность продукт,...,Качество и безопасность продукт,-134.0,Социальные инвестиции,-140.0,разные топики,Безопасность и охрана труда,1,FALSE,FALSE,FALSE


In [31]:
vector = []
labels = []
found_words = []
true_expert_found_words = []

for i, (text, expert_topic) in tqdm(
    enumerate(zip(df["cleaned_text"], df["true_expert"])), total=df["cleaned_text"].shape[0]
):
    tmp = np.zeros(topics_vectors.shape[1])
    if pd.isna(text):
        continue

    if not re_dict.get(expert_topic, False):
        continue

    if len(re_dict[expert_topic].findall(text)) < 5:
        continue

    text_words = {}
    for key, regex in re_dict.items():
        words = regex.findall(text)
        text_words[key] = words
        if key == expert_topic:
            true_expert_found_words.append((key, words))
        for word in words:
            tmp[word2idx[word]] = 1
    vector.append(tmp)
    labels.append(expert_topic)
    found_words.append(text_words)

100%|██████████| 1067/1067 [00:13<00:00, 79.64it/s] 


In [32]:
vector = np.vstack(vector)

In [33]:
w = pd.DataFrame(found_words)
w.head()

,Климат,Энергия,Воздух,Вода,Отходы,Биоразнообразие,Рекультивация земель,Экологичность продукта,Здоровье и благополучие сотрудн,Доступность,...,Волонтерство,Социальные инвестиции,Благотворительность,Отчетность и прозрачность,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие
0,"[безопасность, реализация, территория, безопас...","[требование, деятельность, область, работа, те...",[управление],"[требование, производственный, требование, тре...","[защита, охрана]","[производственный, статус]","[требование, производственный, требование, нео...","[требование, проверка, проверка, требование, п...","[персонал, персонал, работник, работник, работ...","[нарушение, нарушение]",...,"[деятельность, работа, территория, работа, тер...","[деятельность, средство, защита, управление, о...","[реализация, специалист, вопрос, участие, спец...","[проверка, проверка, рамка, проверка, руководи...",[],"[ответственность, государственный, персонал, р...","[ответственность, безопасность, предварительны...","[проверка, проверка, проверка, обучение, обуче...","[ответственность, проведение, проведение, дире...","[деятельность, область, соответствие, область,..."
1,"[контроль, контроль]",[собственный],[],[отчётность],[],[],"[контроль, контроль]","[процедура, независимый]",[],[дополнительный],...,[],"[внешний, финансовый]",[финансовый],"[контроль, внутренний, аудит, контроль, незави...",[обеспечивать],"[независимый, внешний, отчётность]","[внешний, отчётность]",[],[],[дополнительный]
2,"[предотвращение, предотвращение, безопасность,...","[работа, показатель, работа, система, требован...","[уровень, управление, воздействие, воздействие]","[производственный, производственный, требовани...","[развитие, защита, повышение, снижение, воздей...","[производственный, производственный, междунаро...","[производственный, мероприятие, производственн...","[требование, качество, воздействие, воздействи...","[работник, работник]",[],...,"[работа, мероприятие, работа, профессиональный...","[развитие, культура, управление, средство, защ...",[развитие],"[регулярно, руководитель, отчётный, риск, осве...","[следить, риск, риск]","[показатель, руководитель, работник, период, р...","[травматизм, травматизм, безопасность, травмат...",[обучение],[проведение],"[соответствие, риск, риск]"
3,"[оценка, безопасность, показатель, оценка, без...","[работа, показатель, работа]",[],[использовать],[],[],[мероприятие],"[оценка, оценка]",[работник],[],...,"[работа, мероприятие, работа]","[комплексный, профилактика]",[вопрос],[руководитель],[],"[руководитель, показатель, работник]","[безопасность, травматизм, мероприятие, рейтин...",[],[],[]
4,"[безопасность, реализация, безопасность, безоп...","[система, система]","[управление, управление, управление]",[промышленный],"[переход, развитие, охрана]",[международный],[],[],[],[],...,"[проект, программа]","[управление, развитие, культура, управление, о...","[реализация, развитие]","[рамка, аудит]",[],[самооценка],"[безопасность, безопасность, охрана, безопасно...",[],[],[]


In [34]:
p = pd.DataFrame(true_expert_found_words).rename(columns={0: "topic", 1: "words"})
p.head()

,topic,words
0,Безопасность и охрана труда,"[требование, обеспечение, безопасность, подряд..."
1,Отчетность и прозрачность,"[контроль, внутренний, аудит, контроль, незави..."
2,Безопасность и охрана труда,"[травматизм, производственный, травматизм, кул..."
3,Безопасность и охрана труда,"[работа, безопасность, проактивный, травматизм..."
4,Безопасность и охрана труда,"[безопасность, культура безопасность, охрана т..."


In [74]:
print(f"{'Topic':<35} {'Average Word Length':<20} Count")
for topic, val in p.groupby("topic")["words"].apply(list).items():
    print(f"{topic:<35} {sum([len(e) for e in val]) / len(val):05.2f} {len(val)}")

Topic                               Average Word Length  Count
Безопасность и охрана труда         14.84 25
Биоразнообразие                     13.75 4
Благотворительность                 08.00 3
Вовлеченность и мотивация           11.00 1
Вода                                12.05 20
Воздух                              13.00 3
Доступность                         09.00 1
Закупки и антикоррупция             17.00 5
Здоровье и благополучие сотрудн     07.00 3
Качество и безопасность продукт     08.12 8
Климат                              09.27 11
Коренные народы                     12.50 12
Корпоративная культура              14.33 3
Корпоративное управление            06.00 1
Лидерство                           13.14 14
Маркетинг, продажи, реклама         05.00 2
Обучение и развитие                 12.37 19
Оплата труда                        09.83 6
Отходы                              10.50 14
Отчетность и прозрачность           16.08 24
Поставщики в целом                  14.00 3
Профс

In [68]:
matrix = np.zeros(topics_vectors.shape)


def fit(vectors, labels):
    for vector, label in zip(vectors, labels):
        for i in range(matrix.shape[0]):
            if not label2idx.get(label, False):
                continue
            # print(label)
            if i == label2idx[label]:
                matrix[i] += vector
            else:
                matrix[i] -= vector


def test(vectors, labels, debug=False):
    total = 0
    pred_labels = []
    for vector, label in zip(vectors, labels):
        res = np.argmax(vector @ matrix.T)
        # print(np.argmax(vector @ matrix.T))
        total += label == idx2label[res]
        pred_labels.append(idx2label[res])
        if debug and label != idx2label[res]:
            print("expected", label, "result", idx2label[res], vector @ matrix.T)
    return pred_labels, total / len(labels)

In [69]:
for i in tqdm(range(10)):
    fit(vector, labels)
    # test(vector, labels, False)

pred_labels, acc = test(vector, labels, False)

100%|██████████| 10/10 [00:00<00:00, 50.33it/s]

accuracy 0.5481171548117155


In [75]:
np.save("expert.npy", matrix)

In [70]:
# calculate f1, prescion, recall
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print("accuracy",
    accuracy_score(
        labels,
        pred_labels,
    )
)
print("f1",
    f1_score(
        labels,
        pred_labels,
        average="macro",
    )
)
print("precision",
    precision_score(
        labels,
        pred_labels,
        average="macro",
    )
)
print("recall",
    recall_score(
        labels,
        pred_labels,
        average="macro",
    )
)

accuracy 0.5481171548117155
f1 0.2962727923957959
precision 0.38884555018579164
recall 0.293171344784248


/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [113]:
# fit(topics_vectors, list(sorted(topics["topic"].unique())))
# test(topics_vectors, list(sorted(topics["topic"].unique())))

In [14]:
topics_vectors.shape

(40, 3662)

In [13]:
ROWS = 100_000
df = pd.read_csv("paragraphs.csv.zip")  # , usecols=["cleaned_text"], nrows=ROWS
df = df.dropna().reset_index(drop=True)
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

/tmp/ipykernel_2647/610844443.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragraphs.csv.zip")  # , usecols=["cleaned_text"], nrows=ROWS


(2309485, 8)

In [65]:
text = df.loc[724440, "cleaned_text"]
# text = "Доля сотрудников, охваченных коллективными договорами".lower()


def test_algos(text: str, verbouse: bool = False) -> dict[str, str | float | list[str] | None]:
    arr = []
    result = {"text": text}
    vector = np.zeros(matrix.shape[1])

    for key, regex in re_dict.items():
        words = regex.findall(text)
        paragraph_dict = copy.deepcopy(base_dict)
        importance = 0
        paragraph_len = len(text.split())
        base_words = 0
        for word in words:
            paragraph_dict[key][word] = sheets_dict[key][word]
            importance += sheets_dict[key][word] > 1
            vector[word2idx[word]] = 1
            base_words += 1
        # if base_words / paragraph_len < 0.1 or importance < 1:
        #     arr.append((key, 0, 0, []))
        #     continue

        dist = cosine_similarity(X=[list(sheets_dict[key].values())], Y=[list(paragraph_dict[key].values())])[0][0]
        arr.append((copy.deepcopy(key), copy.deepcopy(dist), copy.deepcopy(importance), copy.deepcopy(words)))
        result = {**result, **{f"{key}_dist": dist, f"{key}_importance": importance, f"{key}_words": words}}
    if verbouse:
        print("\nexpert")
    res = vector @ matrix.T
    for idx in np.argsort(res)[::-1]:
        if verbouse:
            print(idx2label[idx], res[idx])
        result[f"{idx2label[idx]}_expert"] = res[idx]

    if verbouse:
        print("\ncos")
        for key, dist, importance, words in sorted(arr, key=lambda x: x[1], reverse=True):
            print(f"{key} расстояние {dist:.4f} важные слова {importance} {copy.deepcopy(set(words))}")
    return result

In [63]:
arr = []
real_topics = [
    "Поставщики. Экология",
    "Этика",
    "Корпоративное управление",
    "Поставщики. Экология",
    "Удовлетворенность потребителей",
    "Доступность",
    "без топика",
    "Работники",
    "Благотворительность" "без топика",
]
for idx in [
    96150,
    596327,
    1032335,
    41290,
    268994,
    137337,
    975491,
    878012,
    601274,
    837328,
]:
    print("\n" + "-" * 20 + "\n" + text)
    text = df.loc[idx, "cleaned_text"]
    arr.append(test_algos(text, True))


--------------------
центральный комиссия по соблюдение норма корпоративный этика и урегулирование конфликт интерес рассматривать вопрос связанный исполнение положение кодекс корпоративный этика компания стандарт корпоративный поведение также связанный урегулирование конфликт интерес

expert

cos
Экология расстояние 0.4264 важные слова 1 {'ответственный', 'экологический'}
Вовлеченность и мотивация расстояние 0.3194 важные слова 1 {'сотрудник'}
Здоровье и благополучие сотрудн расстояние 0.2720 важные слова 1 {'сотрудник'}
Закупки и антикоррупция расстояние 0.2230 важные слова 2 {'организация', 'ответственный', 'субподрядный', 'контроль'}
Обучение и развитие расстояние 0.2150 важные слова 1 {'сотрудник'}
Поставщики в целом расстояние 0.1925 важные слова 0 {'ответственный', 'экологический'}
Биоразнообразие расстояние 0.1909 важные слова 2 {'производственный', 'природоохранный', 'экологический'}
Отчетность и прозрачность расстояние 0.1901 важные слова 2 {'контроль'}
Качество и безопасност

In [79]:
text = df.loc[837328, "cleaned_text"]
print(text)
test_algos(text, True)
a = 0

капитальный ремонт медпункт строительство дом медицинский работник с. абдрахмановый

expert
Здоровье и благополучие сотрудн -26.0
Вовлеченность и мотивация -26.0
Обучение и развитие -26.0
Сервис и коммуникация -26.0
Лидерство -26.0
Социальные инвестиции -30.0
Отходы -30.0
Коренные народы -32.0
Профсоюз и Коллективный договор -32.0
Рекультивация земель -34.0
Этика -36.0
Корпоративная культура -36.0
Климат -36.0
Качество и безопасность продукт -36.0
Доступность -36.0
Здоровье и благополучие -36.0
Закупки и антикоррупция -36.0
Волонтерство -36.0
Воздух -36.0
Вода -36.0
Благотворительность -36.0
Биоразнообразие -36.0
Корпоративное управление -36.0
Наем и увольнение -36.0
Малый и локальный бизнес -36.0
Маркетинг, продажи, реклама -36.0
Энергия -36.0
Оплата труда -36.0
Отчетность и прозрачность -36.0
Персональные данные -36.0
Поставщики в целом -36.0
Права человека -36.0
Работники -36.0
Риски -36.0
Удовлетворенность потребителей -36.0
Устойчивое развитие -36.0
Ценовая политика -36.0
Экологич

In [42]:
re_dict["Этика"].findall(text)

['корпоративный этика',
 'конфликт интерес',
 'кодекс',
 'корпоративный этика',
 'конфликт интерес']

In [39]:
sheets_dict["Этика"]["корпоративный этика"]

1

In [35]:
re_dict["Этика"]

re.compile(r'((?<!\w)горячая линия(?!\w)|(?<!\w)кодекс(?!\w)|(?<!\w)кодекс поведение(?!\w)|(?<!\w)кодекс этика(?!\w)|(?<!\w)комплаенс(?!\w)|(?<!\w)менеджер комплаенс(?!\w)|(?<!\w)менеджер этика(?!\w)|(?<!\w)этика(?!\w)|(?<!\w)этический кодекс(?!\w)|(?<!\w)антикоррупционный(?!\w)|(?<!\w)антикоррупция(?!\w)|(?<!\w)выявленный ущерб(?!\w)|(?<!\w)комиссия по этика(?!\w)|(?<!\w)комитет по этика(?!\w)|(?<!\w)конфликт интерес(?!\w)|(?<!\w)корпоративный этика(?!\w)|(?<!\w)коррупция(?!\w)|(?<!\w)недостаток(?!\w)|(?<!\w)обучение(?!\w)|(?<!\w)проверка(?!\w)|(?<!\w)расследование(?!\w)|(?<!\w)сообщение о недостаток(?!\w)|(?<!\w)сообщение о проблем(?!\w)|(?<!\w)тренинг(?!\w)|(?<!\w)уполномоченный(?!\w)|(?<!\w)уполномоченный этика(?!\w)|(?<!\w)управление этика(?!\w)|(?<!\w)этический (?!\w)|(?<!\w)этический обучение(?!\w)|(?<!\w)амбассадор(?!\w))',
           re.UNICODE)

In [31]:
topics[topics["word"] == "этика"]

,meta,topic,weight,type,word
1154,social,Корпоративная культура,1,0,этика
1946,social,Ценовая политика,1,1,этика
2368,social,Экология,1,1,этика
2453,social,Закупки и антикоррупция,1,1,этика
3084,gov,Лидерство,5,0,этика
3533,gov,Этика,5,0,этика
3598,gov,Корпоративное управление,1,1,этика


In [25]:
q = pd.DataFrame(arr)
q[sorted(q.columns)].to_csv("incorrect_paragraphs.csv", index=False)

In [56]:
def get_scores(text: str) -> dict[str, float | None]:
    paragraph_dict = copy.deepcopy(base_dict)
    paragraph_importance = copy.deepcopy(importance_dict)
    found_words = False
    base_words = {}
    advanced_words = {}
    vector_exp = np.zeros(matrix.shape[1])
    paragraph_len = len(text.split())

    for key, regex in re_dict.items():
        words = regex.findall(text)
        # advanced_words_found = re_advanced_dict[key].findall(text)
        for word in words:
            found_words = True
            weight = sheets_dict[key][word]
            paragraph_dict[key][word] = weight
            paragraph_importance[key] += weight > 1

            vector_exp[word2idx[word]] = 1

        base_words[key] = len(words)
        # advanced_words[key] = len(advanced_words_found)

    if not found_words:
        res = {key: 0 for key in sheets_dict}
        return {**res, **{"max_score_cos": None, "max_topic_cos": None}}

    res = {}
    topic_found = False
    for key, vector in sheets_dict.items():
        if base_words[key] / paragraph_len < 0.1 or paragraph_importance[key] < 1:
            res[key] = 0
            continue
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
        topic_found = True

    if not topic_found:
        res = {key: 0 for key in sheets_dict}
        return {**res, **{"max_score_cos": None, "max_topic_cos": None}}

    paragraph_matrix = vector_exp @ matrix.T
    best_indicies = paragraph_matrix.argsort()[-2:][::-1]
    best_topics = [idx2label[i] for i in best_indicies]
    best_scores = paragraph_matrix[best_indicies]

    res_sorted = sorted(res.items(), key=lambda x: x[1], reverse=True)
    max_value = res_sorted[0]
    second_values = res_sorted[1]
    if second_values[1] == 0:
        second_values = (None, None)

    return {
        **res,
        **{
            "max_topic_cos": max_value[0],
            "max_score_cos": max_value[1],
            "second_max_topic_cos": second_values[0],
            "second_max_score_cos": second_values[1],
            "max_topic_exp": best_topics[0],
            "max_score_exp": best_scores[0],
            "second_max_topic_exp": best_topics[1],
            "second_max_score_exp": best_scores[1],
        },
    }

In [52]:
topics = []

for text in tqdm(df["cleaned_text"].tolist()):
    topics.append(get_scores(text))

  0%|          | 1/2309485 [00:00<19:49:43, 32.35it/s]


In [65]:
# from numba import jit

# @jit(nopython=True)
# def get_scores_jit(text: str) -> dict[str, float | None]:
#     paragraph_dict = copy.deepcopy(base_dict)
#     paragraph_importance = copy.deepcopy(importance_dict)
#     found_words = False
#     base_words = {}
#     advanced_words = {}
#     vector_exp = np.zeros(matrix.shape[1])
#     paragraph_len = len(text.split())
#     res = {}

#     for key, regex in re_dict.items():
#         res[key] = 0
#         words = regex.findall(text)
#         # advanced_words_found = re_advanced_dict[key].findall(text)
#         for word in words:
#             found_words = True
#             weight = sheets_dict[key][word]
#             paragraph_dict[key][word] = weight
#             paragraph_importance[key] += weight > 1

#             vector_exp[word2idx[word]] = 1

#         base_words[key] = len(words)
#         # advanced_words[key] = len(advanced_words_found)

#     if not found_words:
#         res["max_score_cos"] = None
#         res["max_topic_cos"] = None
#         return res

#     res = {}
#     topic_found = False
#     for key, vector in sheets_dict.items():
#         if base_words[key] / paragraph_len < 0.1 or paragraph_importance[key] < 1:
#             continue
#         sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
#         res[key] = sim
#         topic_found = True

#     if not topic_found:
#         res["max_score_cos"] = None
#         res["max_topic_cos"] = None
#         return res

#     paragraph_matrix = vector_exp @ matrix.T
#     best_indicies = paragraph_matrix.argsort()[-2:][::-1]
#     best_topics = [idx2label[i] for i in best_indicies]
#     best_scores = paragraph_matrix[best_indicies]

#     res_sorted = sorted(res.items(), key=lambda x: x[1], reverse=True)
#     max_value = res_sorted[0]
#     second_values = res_sorted[1]
#     if second_values[1] == 0:
#         second_values = (None, None)

#     res["max_topic_cos"] = max_value[0]
#     res["max_score_cos"] = max_value[1]
#     res["second_max_topic_cos"] = second_values[0]
#     res["second_max_score_cos"] = second_values[1]
#     res["max_topic_exp"] = best_topics[0]
#     res["max_score_exp"] = best_scores[0]
#     res["second_max_topic_exp"] = best_topics[1]
#     res["second_max_score_exp"] = best_scores[1]
#     return res

# get_scores_jit("a")
# # %timeit
# get_scores_jit(text)

In [17]:
import pickle

with open("topics.pkl", "wb") as fp:  # Pickling
    pickle.dump(topics, fp)

In [18]:
topics = pd.DataFrame(topics)
topics.to_csv("all_topics.csv.zip")

In [19]:
df = df.join(topics)
df.to_csv("paragrapghs_with_topisc.csv.zip")

In [8]:
df[["max_score", "max_topic", "second_max_score", "second_max_topic"]] = df[
    ["max_topic", "max_score", "second_max_topic", "second_max_score"]
]

In [29]:
n = 30
df.groupby("max_topic_cos").sample(n, replace=True)[
    [
        "company",
        "year",
        "sector",
        "report_type",
        "paragraph",
        "original_text",
        "cleaned_text",
        "max_topic_cos",
        "max_score_cos",
        "second_max_topic_cos",
        "second_max_score_cos",
        "max_topic_exp",
        "max_score_exp",
        "second_max_topic_exp",
        "second_max_score_exp",
    ]
].value_counts(
    "max_topic_cos"
)  # .to_csv(f"paragraph_random_values_{n}.csv.zip")

max_topic_cos
Безопасность и охрана труда        30
Обучение и развитие                30
Отходы                             30
Отчетность и прозрачность          30
Поставщики в целом                 30
Права человека                     30
Профсоюз и Коллективный договор    30
Работники                          30
Рекультивация земель               30
Риски                              30
Сервис и коммуникация              30
Социальные инвестиции              30
Устойчивое развитие                30
Ценовая политика                   30
Экологичность продукта             30
Экология                           30
Энергия                            30
Оплата труда                       30
Наем и увольнение                  30
Биоразнообразие                    30
Маркетинг, продажи, реклама        30
Благотворительность                30
Вода                               30
Воздух                             30
Волонтерство                       30
Доступность                        3

In [2]:
df = pd.read_csv("paragrapghs_with_topisc.csv.zip")
df.head()

/tmp/ipykernel_196006/3911744826.py:1: DtypeWarning: Columns (3,50,51,53,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragrapghs_with_topisc.csv.zip")


,Unnamed: 0.1,Unnamed: 0,company,year,sector,report_type,paragraph,original_text,cleaned_text,Климат,...,Корпоративное управление,Устойчивое развитие,max_score_cos,max_topic_cos,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp
0,0,0.0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,0.0,1 ММК,ммк,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...,0.0,...,0.0,0.0,0.083624,Экология,Работники,0.076472,Профсоюз и Коллективный договор,-49.0,Волонтерство,-49.0
2,2,3.0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,3.0,СОДЕРЖАНИЕ,содержание,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4.0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,5.0,3 ММК,ммк,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(2309485, 57)

In [17]:
df.groupby("max_topic_cos").apply(lambda x : x.nlargest(30, 'max_score_cos'))[
    [
        "company",
        "year",
        "sector",
        "report_type",
        "paragraph",
        "original_text",
        "cleaned_text",
        "max_topic_cos",
        "max_score_cos",
        "second_max_topic_cos",
        "second_max_score_cos",
        "max_topic_exp",
        "max_score_exp",
        "second_max_topic_exp",
        "second_max_score_exp",
    ]
].to_csv("paragraph_max_values_30.csv")#.reset_index()

In [30]:
df.groupby("max_topic").first()  # ["max_score"].first()

,Unnamed: 0,company,year,sector,report_type,paragraph,original_text,cleaned_text,Климат,Энергия,...,Отчетность и прозрачность,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,max_score,second_max_score,second_max_topic
max_topic,,,,,,,,,,,,,,,,,,,,,
0.140988,2244210,АО «ТВЭЛ»,2015,Энергетика,ИО,263.0,Деятельность оказание сопутствующих услуг Пост...,деятельность оказание сопутствовать услуга пос...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.143674,1350987,ПАО «Юнипро»,2018,Энергетика,ИО,4877.0,Предмет Дополнительного соглашения: 1. Стороны...,предмет дополнительный соглашение сторона дого...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.146310,457049,ПАО «Полюс»,2017,Металлургическая и горнодобывающая,ИО,5978.0,Выбытие линий электропередачи По состоянию на ...,выбытие линия электропередача по состояние июн...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.148900,3245071,ПАО «ФСК ЕЭС» («Россети»),2009,Энергетика,ОУР,162.0,3.2. Подход и результативность работ по технич...,подход результативность работа технический обс...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.148900,1350990,ПАО «Юнипро»,2018,Энергетика,ИО,4880.0,"неполном объеме, Заказчик имеет право отказать...",неполный объём заказчик иметь право отказать п...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.845154,3765,ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,2070.0,3.4. Обучение и развитие сотрудников разработк...,обучение развитие сотрудник разработка отдельн...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Вовлеченность и мотивация,Удовлетворенность потребителей,0.493352
0.847566,1323153,АО «СУЭК»,"2019, 2018",Металлургическая и горнодобывающая,ОУР,1621.0,Важнейшим средством продвижения корпоративной ...,важный средство продвижение корпоративный куль...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Вовлеченность и мотивация,Здоровье и благополучие сотрудн,0.571662
0.849970,2947503,Группа ЛСР,2020,Строительство,ОУР,94.0,Работа по усилению практик устойчивого развити...,работа усиление практика устойчивый развитие п...,0.0,0.0,...,0.391381,0,0.0,0.345949,0.0,0.0,0.0,Вовлеченность и мотивация,Обучение и развитие,0.575212


In [ ]:
df.to_csv("paragrapghs_with_topisc.csv")

In [44]:
grouped = pd.DataFrame(df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(" ".join))
grouped.head()

cleaned_text
company     year max_topic                                                                     
Alcoa, Inc. 2004 Антикоррупция                москва белый калитва восток московский офис са...
                 Безопасность и охрана труда          доход европа процент общий доход компания
                 Безопасность продукта        я очень приятно представить ваш внимание первы...
                 Благотворительность          лос летучий органический соединение в тысяча м...
                 Налоги                       возрастание выброс происходить связь применени...

In [45]:
from collections import defaultdict

topics_df = pd.concat([e_topics, s_topics, g_topics])
len_topics = defaultdict(dict)

for idx, row in tqdm(topics_df.iterrows(), total=topics_df.shape[0]):
    if not len_topics[row["topic"]].get(row["type"], False):
        len_topics[row["topic"]][row["type"]] = 0
    len_topics[row["topic"]][row["type"]] += 1

100%|██████████| 4414/4414 [00:00<00:00, 19458.61it/s]


In [46]:
len_topics

defaultdict(dict,
            {'Экология': {1: 51, 2: 8},
             'Климат': {1: 95, 2: 24},
             'Энергия': {1: 52, 2: 11},
             'Воздух': {1: 42},
             'Вода': {1: 61, 2: 8},
             'Отходы': {1: 41, 2: 10},
             'Биоразнообразие': {1: 47},
             'Социально-культурное воздействи': {1: 3},
             'Антикоррупция': {1: 50},
             'Дивиденды и акционеры': {1: 306},
             'Инновации': {1: 104},
             'Лидерство': {1: 371},
             'Отчетность и прозрачность': {1: 530},
             'Предотвращение коррупции': {1: 92},
             'Управление рисками': {1: 214},
             'Эффективность и производительность': {1: 319},
             'Безопасность и охрана труда': {1: 349},
             'Безопасность продукта': {1: 50},
             'Благотворительность': {1: 259},
             'Инвестиции и капитальные вложения': {1: 370},
             'Налоги': {1: 81},
             'Обучение и развитие': {1: 50},
        

In [47]:
from __future__ import annotations


def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent = count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [55]:
scores = []

for (company, year, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

100%|██████████| 8788/8788 [01:05<00:00, 135.19it/s]


In [57]:
grouped["score"] = scores
grouped.head(20)

cleaned_text  \
company     year       max_topic                                                                              
Alcoa, Inc. 2004       Антикоррупция                      москва белый калитва восток московский офис са...   
                       Безопасность и охрана труда                доход европа процент общий доход компания   
                       Безопасность продукта              я очень приятно представить ваш внимание первы...   
                       Благотворительность                лос летучий органический соединение в тысяча м...   
                       Налоги                             возрастание выброс происходить связь применени...   
                       Отношения с потребителями                 белый калитва любучан восток москва самара   
                       Отношения с работниками            вступление выброс загрязнять вещество атмосфер...   
                       Отходы                             показатель здоровье работник безопасность труд...   
                       Предотвращение коррупции           наш преимущество заключаться производить очень...   
                       Профсоюзы и коллективные договоры  показатель экологический безопасность для комп...   
                       Управление рисками                 отчёт устойчивый развитие год отчёт ход выполн...   
BIOCAD      2019, 2018 Антикоррупция                      соответствие отчёт руководство прямой речь пре...   
                       Безопасность и охрана труда        экопроект павел яко вл быть в ранний разработк...   
                       Безопасность продукта              содержание устойчивый развитие стратегия приор...   
                       Благотворительность                август россия мир уровень вовлечённость сотруд...   
                       Вода                               характер собственность форма существенный изме...   
                       Дивиденды и акционеры              сокращать отход производство упаковка снижать ...   
                       Инвестиции и капитальные вложения  самый продавать препарат одержать победа суд п...   
                       Инновации                          генферон® ацеллбия® альгерон® эфлейра® млн руб...   
                       Климат                             о компания отход удаться ускорить синтезирован...   

                                                          score  
company     year       max_topic                                 
Alcoa, Inc. 2004       Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Налоги                                 5  
                       Отношения с потребителями              5  
                       Отношения с работниками                5  
                       Отходы                                 7  
                       Предотвращение коррупции               5  
                       Профсоюзы и коллективные договоры      5  
                       Управление рисками                     5  
BIOCAD      2019, 2018 Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Вода                                   7  
                       Дивиденды и акционеры                  5  
                       Инвестиции и капитальные вложения      5  
                       Инновации                              5  
                       Климат                                 7

In [58]:
grouped.to_csv("grouped_paragraphs.csv.zip")